# 1.Yolov5的数据准备

主要是在指定的路径下放好训练所需的数据

In [ ]:
ENV = "Meg"
datadir = '/home/megstudio/workspace/competedata'
outputdir = '/home/megstudio/workspace'
archivedir = '/home/megstudio/workspace/archive'

In [ ]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
import time
from IPython.display import Image, clear_output  # to display images
from os import listdir
from os.path import isfile, join
import yaml
from sklearn.model_selection import GroupKFold
clear_output()

In [ ]:
class CFG:
    suffix = "2400"
    batch_size = 64
    epochs = 50
    image_size = 672 #记得改文件名  siim_yolo_label_txt
    model_pt = f"{outputdir}/yolov5-master/models/yolov5_kaggle.yaml"
    
    debug = False

In [ ]:
# 切分fold
train_df = pd.read_csv(f'{datadir}/train_image_level.csv')
fold = 0
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups=train_df.StudyInstanceUID.tolist())):
    train_df.loc[val_idx, 'fold'] = fold

train_df['image_path'] = f'{datadir}/train{CFG.image_size}png/train/' + train_df["id"].str.replace("_image","") + '.png'
val_files   = list(train_df[train_df.fold==fold].image_path.unique())
train_files = list(train_df[train_df.fold!=fold].image_path.unique())
len(train_files), len(val_files)

In [ ]:
# 将含label的txt文件，划分到train和val文件夹下.
label_dir = f'{datadir}/siim_yolo_label_txt'
os.makedirs(f'{datadir}/siim_yolo_dataset/labels/train', exist_ok = True)
os.makedirs(f'{datadir}/siim_yolo_dataset/labels/val', exist_ok = True)
os.makedirs(f'{datadir}/siim_yolo_dataset/images/train', exist_ok = True)
os.makedirs(f'{datadir}/siim_yolo_dataset/images/val', exist_ok = True)
for file in tqdm(train_files):
    shutil.copy(file, f'{datadir}/siim_yolo_dataset/images/train')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), f'{datadir}/siim_yolo_dataset/labels/train')
    
for file in tqdm(val_files):
    shutil.copy(file, f'{datadir}/siim_yolo_dataset/images/val')
    filename = file.split('/')[-1].split('.')[0]
    shutil.copy(os.path.join(label_dir, filename+'.txt'), f'{datadir}/siim_yolo_dataset/labels/val')

In [ ]:
# 新建一个data yaml文件
classes = ['0. opacity']

with open(join( datadir , 'siim_yolo_dataset/train.txt'), 'w') as f:
    for path in glob(f'{datadir}/siim_yolo_dataset/images/train/*'):
        f.write(path+'\n')
            
with open(join( datadir , 'siim_yolo_dataset/val.txt'), 'w') as f:
    for path in glob(f'{datadir}/siim_yolo_dataset/images/val/*'):
        f.write(path+'\n')

data = dict(
    train =  join( datadir , 'siim_yolo_dataset/train.txt'),
    val   =  join( datadir , 'siim_yolo_dataset/val.txt' ),
    nc    = 1,
    names = classes
    )

with open(join( datadir , 'siim_yolo_dataset/siim_dataset.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(join( datadir , 'siim_yolo_dataset/siim_dataset.yaml'), 'r')
print(f.read())

# 2.Yolov5的训练

In [1]:
%cd workspace/yolov5-master

[WinError 3] 系统找不到指定的路径。: 'workspace/yolov5'
C:\Users\xm\jupyternotebook


In [ ]:
!WANDB_MODE="dryrun" python train.py --img $CFG.image_size --batch-size $CFG.batch_size --worker 16 --epochs $CFG.epochs --data /home/megstudio/workspace/competedata/siim_yolo_dataset/siim_dataset.yaml --hyp /home/megstudio/workspace/hyp.scratch.yaml --cfg $CFG.model_pt --weights '' --cache